In [1]:
%load_ext autoreload
%autoreload 2

%config IPython.sys.argv=['-Xfrozen_modules=off']

# 2. Sweep of Geometries

The second part of this repository is to help users run a large simulation sweep over various geometries (with the hopes that the community will contribute their finding as well 😊)

We implemented sweeping of geometires with a `QSweeper` class.

# Example - Qubit Only

## Outline of Design
In this example, we'll be simulating only an XMON, represented in Qiskit Metal as the `TransmonCross` class. 

In [2]:
from qiskit_metal.designs.design_multiplanar import MultiPlanar

import numpy as np
import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, open_docs

design = MultiPlanar(metadata={},
                     overwrite_enabled=True)

# Design variables
design._chips.main.size.size_x = '5mm'
design._chips.main.size.size_y = '5mm'


# Check if layer data is useable, if so launch GUI
ls_unique = design.ls.is_layer_data_unique()
if (ls_unique != True):
    raise ValueError('Layer data in `MultiPlanar` design is not unique')

# gui = MetalGUI(design)

In [3]:
metal.__version__

'0.1.5'

In [4]:
# Transmon
from qiskit_metal.qlibrary.qubits.transmon_cross import TransmonCross

xmon_options = Dict(
    connection_pads=Dict(
        readout = Dict(connector_location = '180')),
)


# Create a new Transmon Cross object with name 'Q1'
TransmonCross(design, 'transmon', options=xmon_options)

# gui.rebuild()  # rebuild the design and plot
# gui.autoscale() #resize GUI to see QComponent

name:    transmon
class:   TransmonCross         
options: 
  'pos_x'             : '0.0um',                      
  'pos_y'             : '0.0um',                      
  'orientation'       : '0.0',                        
  'chip'              : 'main',                       
  'layer'             : '1',                          
  'connection_pads'   : {
       'readout'           : {
            'connector_type'    : '0',                          
            'claw_length'       : '30um',                       
            'ground_spacing'    : '5um',                        
            'claw_width'        : '10um',                       
            'claw_gap'          : '6um',                        
            'claw_cpw_length'   : '40um',                       
            'claw_cpw_width'    : '10um',                       
            'connector_location': '180',                        
                             },
                        },
  'cross_width'       : '20um

## Simulation Setup
### Static variables in simulation

In [5]:
from qiskit_metal.renderers.renderer_ansys_pyaedt.hfss_renderer_eigenmode_aedt import QHFSSEigenmodePyaedt
import pyEPR as epr

In [6]:
em1_aedt = QHFSSEigenmodePyaedt(design, 
                                'Qubit_Sabrina_proj',
                                'Qubit_Sabrina_design',
                                initiate=True)
hfss_aedt = em1_aedt.current_app

pyaedt INFO: using existing logger.


INFO:Global:using existing logger.


pyaedt INFO: Launching PyAEDT outside AEDT with CPython and PythonNET.


INFO:Global:Launching PyAEDT outside AEDT with CPython and PythonNET.


pyaedt INFO: AEDT installation Path D:\Program Files\ANSYS\AnsysEM21.1\Win64.


...

INFO:Global:Design Loaded


pyaedt INFO: Successfully loaded project materials !


INFO:Global:Successfully loaded project materials !


pyaedt INFO: Materials Loaded


INFO:Global:Materials Loaded


### Dynamic variables

As the simulation goes on, we need it to re-render the design into ANSYS, delete old simulation sets (found in `ANSYS -> Setup`), update junctions in `pyEPR`, and extract current results...

**Note:** for this example simulation, I've adjusted the `Setup` to run quickly. For good results, I'd recommend changing `MaximumPasses -> 30+`

In [7]:
def render_design():
    em1_aedt.render_design()

In [8]:
def run_simulation():
    #### Computer's variables
    num_cores=16
    num_gpu=1
    
    #### Setup Name
    setup_name = 'TransmonSetup'
    
    # Change Silicon to Ultracold Silicon
    hfss_aedt.materials['silicon'].permittivity = 11.45
    
    # Add project variables
    add_project_level_vars(em1_aedt)
    
    # Clear setups
    if len(hfss_aedt.setups) != 0:
        hfss_aedt.setups[0].delete()
    
    # Add Setup
    setup_options = dict(
        name= setup_name,
        MinimumFrequency = 3,  # GHz
        NumModes= 1,
        MaxDeltaFreq = 0.1,
        MaximumPasses= 1,
        MinimumPasses= None,
        MinimumConvergedPasses= 2,
        PercentRefinement= 30,
        BasisOrder= None
    )
    
    em1_aedt.add_hfss_em_setup(**setup_options)
    
    hfss_aedt.analyze_setup(name=setup_name, 
                            num_cores=num_cores, 
                            num_gpu=num_gpu)
    
    # Make silicon ultra cold
    hfss_aedt.materials['silicon'].permittivity = 11.45

def add_project_level_vars(renderer):
    """
    Finds all names, inductances, and capacitances of Josephson Junctions rendered into ANSYS.

    Args:
        renderer (QHFSSEigenmodePyaedt)
    """
    # Get all josephson junctions from rendered components table
    geom_table = renderer.path_poly_and_junction_with_valid_comps
    all_jjs = geom_table.loc[geom_table['name'].str.contains('rect_jj')]
    all_jjs = all_jjs.reset_index(drop=True)

    for i, row in all_jjs.iterrows():
        ### Parsing Data ###
        component = str(row['component'])
        name = str(row['name'])
        inductance = row['aedt_hfss_inductance']  # Lj in Henries
        capacitance = row['aedt_hfss_capacitance']  # Cj in Farads

        # Get ANSYS > Model > Sheet corresponding to JJs
        rect_name = 'JJ_rect_Lj_' + component + '_' + name

        # Get ANSYS > Model > Lines corresponding to JJs
        line_name = 'JJ_Lj_' + component + '_' + name + '_'

        ### Appending data ###
        # Add global Lj and Cj variables to ANSYS (for EPR analysis)
        ansys_Lj_name = f'Lj_{i}'
        ansys_Cj_name = f'Cj_{i}'

        renderer.set_variable(ansys_Lj_name, str(inductance * 1E9) + 'nH')
        renderer.set_variable(ansys_Cj_name, str(capacitance * 1E15) + 'fF')

def run_analysis():
    '''
    After finishing simulation, analyze results
    '''
    em1_aedt.run_epr()
    
    # Extract observables of int
    observables = extract_observables(em1_aedt.epr_quantum_analysis)
    return observables



def extract_observables(epra, 
                        cos_trunc: int = 10, 
                        fock_trunc: int = 15):
    '''
    Extract observables from EPR quantum analysis
    
    Args:
        epra (epr.QuantumAnalysis): Raw EPR Quanutm analysis class
        cos_trunc (int, optional): amount of terms to consider in cosine expansion
        fock_trunc (int, optional): size of fock space before truncation
        
    Returns:
        package (dict): simulated observables
    '''
    
    # Extraction of variables
    omegas = epra.get_frequencies()
    chis = epra.get_chis()
    other_data = epra.data
    
    package = dict(qubit_freq_MHz = omegas['0'][0],
                   qubit_anharmonicity_MHz = chis[0][0],
                   other_data = str(other_data))
    
    return package

# Sweep Over Geometires

To simulate a large range of geometires, we use `QSweeper.run_single_component_sweep`
```
Runs custom_analysis() for all combinations of the options and values in the `parameters` dictionary.

Inputs:
* component_name (str) - The name of the component to run the sweep on.
* parameters (dict) - A dictionary of options and their corresponding values. 
    The keys are the options (strings), and the values are lists of floats.
* custom_analysis (func (QAnalysis) -> dict) - Create a custom analyzer to parse data
* parameters_slice (slice) - If sweep fails, tell it where to start again from
    Example:
    slice(40,)
* save_path (str, optional) - save data path associated from sweep
* kwargs - parameters associated w/ QAnalysis.run()

Output:
* Librarian (QLibrarian)
```

I wanted the `parameters` obj to mimic the way you modify geometires natively in Qiskit Metal. 
So you will define the parameters with the same type of recursion in your `QComponent`. The only difference is you will end it with a list of values to sweep over, NOT just the value. I understand this was a little confusing, so I've included an example below.

```
Example:
An acceptable input would be:

`parameters = {
    'connection_pads' : {
        'readout' : {
            'claw_length' : ['10um', '20um'],
        }
    }
    'cross_length': ['30um', '40um', '50um']
}`

This method will call 
`custom_analysis()` 6 times, where `design.components[component_name].options` gets update to:
1. {'connection_pads' : {'readout': {'claw_length : '10um'}}, 'cross_length' : '30um'}
2. {'connection_pads' : {'readout': {'claw_length : '20um'}}, 'cross_length' : '30um'}
3. {'connection_pads' : {'readout': {'claw_length : '10um'}}, 'cross_length' : '40um'}
4. {'connection_pads' : {'readout': {'claw_length : '20um'}}, 'cross_length' : '40um'}
5. {'connection_pads' : {'readout': {'claw_length : '10um'}}, 'cross_length' : '50um'}
6. {'connection_pads' : {'readout': {'claw_length : '20um'}}, 'cross_length' : '50um'}

```

In [9]:
def run_single_design(): # this will be used as `custom_analysis`

    render_design()
    run_simulation()
    package = run_analysis()
    return package

In [10]:
from metal_library.sweeper import QSweeper
sweeper = QSweeper(design)

In [11]:
sweep_parameters = dict(
  connection_pads = dict(
    readout = dict(
      connector_type = '0', 
      claw_length = ['185um', '195um', '205um', '215um'],
      ground_spacing = ['4um', '5um', '7um', '9um', '10um'], 
      claw_gap = ['5.1um'], 
      claw_width = ['10um', '15um']
      )
    ),
  cross_width = ['30um'],
  cross_length = ['185um', '195um', '205um', '215um', '225um', '235um'],
  cross_gap = ['29um'],
  aedt_hfss_inductance = [9.686E-9, 10.471E-9, 11.268E-9, 12.679-9, 13.816]
)

my_librarian = sweeper.run_single_component_sweep(component_name='transmon',
                                                  parameters = sweep_parameters,
                                                  custom_analysis = run_single_design,
                                                  parameters_slice=slice(1,2),
                                                  save_path="./Tutorial2_QubitOnly.csv")

  0%|          | 0/1 [00:00<?, ?it/s]

pyaedt INFO: Project Qubit_Sabrina_proj set to active.


INFO:Global:Project Qubit_Sabrina_proj set to active.


pyaedt INFO: Aedt Objects initialized


INFO:Global:Aedt Objects initialized


...

###### Numerical diagonalization

,,0
variation,,
0,0,78.6


Warning! 51 COM references still alive
Ansys will likely refuse to shut down


100%|██████████| 1/1 [00:17<00:00, 17.36s/it]

Simulated and logged configuration: {'connection_pads': {'readout': {'connector_type': '0', 'claw_length': '185um', 'ground_spacing': '4um', 'claw_gap': '5.1um', 'claw_width': '10um'}}, 'cross_width': '30um', 'cross_length': '185um', 'cross_gap': '29um', 'aedt_hfss_inductance': 1.0471e-08}
